In [ ]:
# 브랜드명, 가격들, 상품명 가져오기 

In [1]:
from selenium import webdriver
import chromedriver_autoinstaller
import time 
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
print(chrome_ver)


109


In [11]:
try:
    crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')
except:
    chromedriver_autoinstaller.insrall(True)
    crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')
crawler.implicitly_wait(10)  # timesleep이랑 비슷한 명령어

# 올리브영 메인화면
crawler.get('https://www.oliveyoung.co.kr/store/main/main.do?oy=0') 

C:\Users\admin\AppData\Local\Temp/ipykernel_1788/3323440318.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')


In [12]:
# 올리브영에서 카테고리 클릭
crawler.find_element('xpath', '//*[@id="btnGnbOpen"]').click()
# 스킨케어 클릭
crawler.find_element('xpath', '//*[@id="gnbAllMenu"]/ul/li[1]/div[1]/p[1]/a').click()
# 토너/로션/올인원 클릭
crawler.find_element('xpath', '//*[@id="Contents"]/div/div[1]/ul/li[1]/a/span').click()
# 인기순 클릭
crawler.find_element('xpath', '//*[@id="Contents"]/div[5]/div[1]/ul/li[1]/a').click()
# 별점있는 카테고리 클릭
crawler.find_element('xpath', '//*[@id="Contents"]/div[5]/div[3]/button[2]').click()

In [ ]:
## 원본

brands = []
pnames = []
pprices = []
prices = []
stars = []

# 페이지 넘기기 : 페이지 넘어가면서 스크롤바 조정
for i in range(1, 35):
    crawler.get('https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx={0}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010008_Small'.format(i))
                
    while True:
        bh = crawler.execute_script('return document.body.scrollHeight') # 브라우저 상의 처음 높이 
        print(bh) 
        time.sleep(4)
        crawler.execute_script('window.scrollTo(0, document.body.scrollHeight)') # 스크롤 내리기
        time.sleep(2)
        ah = crawler.execute_script('return document.body.scrollHeight') 
        if ah == bh:
            break
        bh = ah
    infos = crawler.find_elements(By.CSS_SELECTOR, '.prd_info')
    for info in infos:
        try:
            brand = info.find_element(By.CSS_SELECTOR,'.tx_brand').text
            brands.append(brand)
            pname = info.find_element(By.CSS_SELECTOR,'.tx_name').text
            pnames.append(pname)
            pprice = info.find_element(By.CSS_SELECTOR,'.tx_org').text  #(원가) : 가격(원가+할인: 같이 추출, 이유: 누락되면 같이 누락됌)은 전처리 과정에서 슬라이싱 하기
            pprices.append(pprice)
            price = info.find_element(By.CSS_SELECTOR,'.tx_cur').text  # 현재가 : 항상있는 것(제외하면 안됌)
            prices.append(price)
            star = info.find_element(By.CSS_SELECTOR,'.review_point').text
            stars.append(star)

        except:
            price = info.find_element(By.CSS_SELECTOR,'.prd_price').text  
            prices.append(price)
            pprices.append("No Sale")
            print("No Sale")  # 할인 안하는 제품

crawler.close()

3848
No Sale
No Sale
3848
No Sale
No Sale
No Sale
No Sale
No Sale
No Sale
3848
No Sale


In [ ]:
# 데이터 프레임 key들끼리 행 길이 같아야 함
ddf = pd.DataFrame({'brand':brands, 'pprice':pprices, 'price':prices, 'pname':pnames})
print(ddf)

In [ ]:
# 데이터 프레임 엑셀로 저장, utf-8-isg, csv는 sep 은 ,로 구분 => 파일 안깨짐
ddf.to_csv('./best_cosmetic22.csv', sep = ',', encoding = 'utf-8-sig')